# what is wrong with my labeled blobs?

In [ ]:
import cv2
import numpy as np
import pandas as pd
import Args
import pickle

data = pd.read_pickle('output.labeled.bz2')

model = pickle.load(open('models/LR1.pkl', 'rb'))

In [ ]:
data.describe()

In [ ]:
# %load LabelBlobs.py
'''
Label the blobs using a previously trained 

Gary Bishop July 2018
'''

import cv2
import numpy as np
import pandas as pd
import Args
import pickle

args = Args.Parse(
    inblobs='output.blobs.bz2',
    outblobs='output.labeled.bz2',
    model='models/LR1.pkl'
)

data = pd.read_pickle(args.inblobs)

In [ ]:
model = pickle.load(open(args.model, 'rb'))

In [ ]:
def isBlue(im):
    mblue = np.array([60.4, -12.2, -35.7], dtype=np.float32)
    sblue = np.array([4.1, 3.2, 8.5], dtype=np.float32)
    d2 = np.sum((im - mblue)**2 / sblue**2, axis=2)
    return np.exp(-d2 / 20)


def circularity(contour):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    area = cv2.contourArea(contour)
    result = 4 * np.pi * (area / perimeter ** 2)
    return result


def isCircular(contour, hull=False):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    if hull:
        contour = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    circularity = 4 * np.pi * (area / perimeter ** 2)
    return 0.7 <= circularity <= 1.2
    # return 0.5 <= circularity <= 1.4


def isInside(contour, size):
    r = np.zeros(size[:2], dtype=np.uint8)
    cv2.drawContours(r, [contour - np.min(contour, axis=0)],
                     -1, 255, cv2.FILLED)
    return r == 255


def getFeatures(df):
    result = []
    for row in df.itertuples():
        plt.figure()
        plt.imshow(row.pixels)
        lab = cv2.cvtColor(np.float32(row.pixels) / 255, cv2.COLOR_RGB2LAB)
        bim = isBlue(lab)
        plt.figure()
        plt.imshow(bim, cmap='gray')
        h, w = bim.shape
        circ = circularity(row.contour)
        peri = cv2.arcLength(row.contour, True)
        area = cv2.contourArea(row.contour)
        roughness = cv2.arcLength(cv2.convexHull(row.contour), True) / peri
        isin = isInside(row.contour, bim.shape)
        fracin = np.sum(isin) / (h * w)
        std0 = np.std(bim[isin])
        std1 = np.std(cv2.dilate(bim, np.ones((3, 3)))[isin])
        print(circ, peri, area, roughness, fracin, std0, std1)
        result.append([circ, peri, area, roughness, fracin, std0, std1])
    return np.array(result)


features = getFeatures(data[8:9])

labels = model.predict(features)
labels

#data.to_pickle(args.outblobs)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl

plt.imshow(data.iloc[8].pixels)
print(data.iloc[8].fnos)

In [ ]:
data.at[1892, 'isdot']

In [ ]:
edited = pd.read_pickle('output.edited.bz2')